In [ ]:
!cp ../input/hungry-geese-is-a-nlp-problem-part-2/model.h5 ./model.h5

In [ ]:
%%writefile main.py
import tensorflow as tf
import os
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dropout, Dense

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1, training=True):
        super(EncoderLayer, self).__init__()
        self.training = training
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x):
        attn_output = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=self.training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=self.training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class Net(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, rate=0.1, training=True):
        super(Net, self).__init__()
        self.training = training
        self.d_model = d_model
        self.num_layers = num_layers
        
        self.emb = Embedding(input_dim=50, output_dim=128)
        self.pos_emb = Embedding(input_dim=77, output_dim=128) # relative positional embedding
        self.cls_token_emb = Embedding(input_dim=1, output_dim=256) # class token

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate, self.training)
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
        bs = 128 if self.training else 1
        self.token = tf.zeros((bs,1),dtype=tf.int64)
        self.policy_head = Dense(4, activation=None, use_bias=False)
        
    def call(self, sentence, positions):
        h = self.emb(sentence)
        h_pos = self.pos_emb(positions)
        h = tf.concat((h_pos,h), 2)
        
        h_token = self.cls_token_emb(self.token)
        h = tf.concat((h_token,h), 1)
        
        h = self.dropout(h, training=self.training)
        
        for i in range(self.num_layers):
            h = self.enc_layers[i](h)
        
        action_token_h = h[:,0]

        return self.policy_head(action_token_h)

model_inf = Net(num_layers=6, d_model=256, num_heads=8, dff=2048, training=False)
model_inf(tf.zeros((1,77),dtype=tf.int64),tf.zeros((1,77),dtype=tf.int64))

tgz_agent_path = '/kaggle_simulations/agent/'
normal_agent_path = '/kaggle/working'
model_name = "model"

file_name = os.path.join(normal_agent_path, f'{model_name}.h5')
if not os.path.exists(file_name):
    file_name = os.path.join(tgz_agent_path, f'{model_name}.h5')
model_inf.load_weights(file_name)

import numpy as np
from kaggle_environments.envs.hungry_geese.hungry_geese import Action

def find_last_player_action(new_obs,previous_obs=None):
    last_actions = ["NORTH"]*4
    
    if previous_obs is None:
        return last_actions
    
    for p in range(4):
        if not len(previous_obs["geese"][p]) or not len(new_obs["geese"][p]):
            continue
        
        previous_head = previous_obs["geese"][p][0]
        previous_head_x = previous_head % 11
        previous_head_y = previous_head // 11
        
        new_head = new_obs["geese"][p][0]
        new_head_x = new_head % 11
        new_head_y = new_head // 11
        
        if previous_head_x < new_head_x:
            if previous_head_x == 0 and new_head_x == 10:
                last_actions[p] = "WEST"
            else:
                last_actions[p] = "EAST"
        elif previous_head_x > new_head_x:
            if previous_head_x == 10 and new_head_x == 0:
                last_actions[p] = "EAST"
            else:
                last_actions[p] = "WEST"
        elif previous_head_y < new_head_y:
            if previous_head_y == 0 and new_head_y == 6:
                last_actions[p] = "NORTH"
            else:
                last_actions[p] = "SOUTH"
        elif previous_head_y > new_head_y:
            if previous_head_y == 6 and new_head_y == 0:
                last_actions[p] = "SOUTH"
            else:
                last_actions[p] = "NORTH"
            
    return last_actions

def preprocess_map_obs(obs, previous_obs=None, p=None):
    if p is None:
        p = 0
        
    relativ_center = obs['geese'][p][0]
    relativ_poss = np.roll(np.arange(77), relativ_center)
    
    previous_actions = find_last_player_action(obs,previous_obs)
    sentence = []
    positions = []
    for pp in range(4):
        real_player_index = pp
        player_index = (pp - p) % 4
        geese_length = len(obs['geese'][real_player_index])
        for goose_body_position, goose_board_position in enumerate(obs['geese'][real_player_index]):
            if goose_body_position == 0:
                body_part = 0
            elif goose_body_position == (geese_length-1):
                body_part = 2
            else:
                body_part = 1

            last_action = Action[previous_actions[pp]].value

            index_player = 3*4 * player_index
            index_bodypart = 4*body_part
            
            word_unique_index = index_player + index_bodypart + last_action + 1
            sentence.append(word_unique_index)
            
            position = relativ_poss[goose_board_position]
            positions.append(position)
            

    for food_board_position in obs['food']:
        word_unique_index = 47 + 1 + 1
        sentence.append(word_unique_index)
        
        position = relativ_poss[food_board_position]
        positions.append(position)
        
    left_positions = set(range(0,77))-set(positions)
    
    positions = positions + list(left_positions)
    sentence = sentence + [0]*(77-len(sentence))
        
    return sentence, positions

obss = []
action_mapping = ['NORTH', 'SOUTH', 'WEST', 'EAST']

def agent(obs, _):
    previous_obs = obss[-1] if len(obss) else None
    sentence, positions = preprocess_map_obs(obs, previous_obs=previous_obs, p=obs['index'])

    sentence = tf.convert_to_tensor(sentence, dtype=tf.int64)
    positions = tf.convert_to_tensor(positions, dtype=tf.int64)
    preds = model_inf.call(tf.expand_dims(sentence,0),tf.expand_dims(positions,0))
    pred = tf.math.argmax(preds,1).numpy()[0]
    
    action = action_mapping[pred]
    
    obss.append(obs)
    return action

In [ ]:
from kaggle_environments import make
env = make("hungry_geese", debug=True)

env.reset()
env.run(['main.py', 'main.py', 'main.py', 'main.py'])
env.render(mode="ipython", width=800, height=700)

In [ ]:
!tar cvzf submission.tar.gz main.py model.h5